In [1]:
'''
MAML with uncertainty
'''

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns 
from math import pi as PI
import random
import torch.nn as nn
import torch
import random
from higher import innerloop_ctx
import warnings
import importlib

warnings.filterwarnings("ignore")

#Set random seeds for reproducibility of results 
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# set GPU or CPU depending on available hardware
# help from: https://stackoverflow.com/questions/46704352/porting-pytorch-code-from-cpu-to-gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Available device: {device}")

if device == "cuda:0": 
  # set default so all tensors are on GPU, if available
  # help from: https://stackoverflow.com/questions/46704352/porting-pytorch-code-from-cpu-to-gpu
  torch.set_default_tensor_type('torch.cuda.FloatTensor')

# import backbone model, dataset, and code utils
from models import Neural_Network, Prob_Neural_Network
from constants import *
from utils import *
import analysis_utils
from data import *

Available device: cpu
Available device: cpu


In [2]:
'''
Create dataset
'''
# specify the number of tasks to sample per meta-set
# note: we end up sampling tasks at random, so sizes are not particularly relevant
# artifact of the way we structured the dataset earlier 
meta_train_size=10000
meta_val_size=1000
meta_test_size=1000
meta_train_eval_size = 20

dataset = RegressionDomain(amp_min=amp_min, amp_max=amp_max, 
                           phase_min=phase_min, phase_max=phase_max, 
                           train_size=meta_train_size, val_size=meta_val_size, test_size=meta_test_size)

meta_val_set = dataset.get_meta_val_batch()
meta_test_set = dataset.get_meta_test_batch()

In [15]:
'''
Create custom probabilistic loss
Help from: 
- https://discuss.pytorch.org/t/custom-loss-functions/29387
- https://pytorch.org/docs/stable/distributions.html#normal
- https://towardsdatascience.com/predicting-probability-distributions-using-neural-networks-abef7db10eac
'''
from torch.distributions.normal import Normal

def log_prob_loss(output, target): 
    # follows format for custom loss functions in pytorch from: https://discuss.pytorch.org/t/custom-loss-functions/29387
    # here, we assume output are the mean and sigma per x
    mu, sigma = output
    sigma += 1e-15 # avoid 0.0 issues
    predicted_gaussians = Normal(mu, sigma)
    return -predicted_gaussians.log_prob(target)

criterion = log_prob_loss

In [16]:
'''
Handling computation graphs and second-order backprop help and partial inspiration from: 
- https://discuss.pytorch.org/t/how-to-save-computation-graph-of-a-gradient/128286/2 
- https://discuss.pytorch.org/t/when-do-i-use-create-graph-in-autograd-grad/32853/3 
- https://lucainiaoge.github.io/download/PyTorch-create_graph-is-true_Tutorial_and_Example.pdf
- https://www.youtube.com/watch?v=IkDw22a8BDE
- https://discuss.pytorch.org/t/how-to-manually-update-network-parameters-while-keeping-track-of-its-computational-graph/131642/2
- https://discuss.pytorch.org/t/how-to-calculate-2nd-derivative-of-a-likelihood-function/15085/3
- https://pytorch.org/tutorials/recipes/recipes/zeroing_out_gradients.html
- https://higher.readthedocs.io/en/latest/toplevel.html

Neural network configuration and helper class functions copied directly from 
-https://github.com/AdrienLE/ANIML/blob/master/ANIML.ipynb

Note, different ways to refer to the task-specific vs. meta/aggregate updates to the parameters
Sometimes called "inner" and "outer" loop, respectively
Here, refered to as "task_specific" and "agg"/meta" (the latter, for consistency w/ ocariz code)
'''


#Instantiate the model network
model = Prob_Neural_Network()
# move to the current device (GPU or CPU)
# help from: https://stackoverflow.com/questions/46704352/porting-pytorch-code-from-cpu-to-gpu
model.to(device)

N = 1 # number of inner loop steps (notation from: https://www.bayeswatch.com/2018/11/30/HTYM/)
K = 10 # number of samples to draw from the task

#Used to store the validation losses
metaLosses = []
metaValLosses = []

#Meta-optimizer for the outer loop
meta_optimizer = torch.optim.Adam(model.parameters(), lr = lr_meta)

#Inner optimizer, we were doing this by hand previously
inner_loop_optimizer = torch.optim.SGD(model.parameters(), lr = lr_task_specific)

for epoch in range(num_epochs):
    # store loss over all tasks to then do a large meta-level update of initial params
    # idea/help from video: https://www.youtube.com/watch?v=IkDw22a8BDE
    meta_loss = None
    
    waves = dataset.get_meta_train_batch(task_batch_size=T)
    
    #Loop through all of the tasks
    for i, T_i in enumerate(waves): 
        train_eval_info = task_specific_train_and_eval(model, T_i, inner_loop_optimizer, criterion, K=K, N=N)
        held_out_task_specific_loss = train_eval_info[0]
        if meta_loss is None: 
            meta_loss = held_out_task_specific_loss
        else:
            meta_loss += held_out_task_specific_loss
            
    meta_optimizer.zero_grad()
    meta_loss /= T
    meta_loss.backward()
    meta_optimizer.step()
    metaLosses.append(meta_loss.item())
    
    # validation 
    val_wave = dataset.get_meta_val_batch(task_batch_size=1)[0]
    val_train_eval_info = task_specific_train_and_eval(model, val_wave, inner_loop_optimizer, criterion, K=K, N=N)
    val_loss = val_train_eval_info[0]
    metaValLosses.append(val_loss.item())
    
    if epoch % printing_step == 0:
        print("Iter = ", epoch, " Current Loss", np.mean(metaLosses), " Val Loss: ", np.mean(metaValLosses))
        # saving model help from: 
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html
        torch.save(model.state_dict(), "sample_maml_model.pt")

RuntimeError: grad can be implicitly created only for scalar outputs

In [3]:
# help from torch distributions documentation: https://pytorch.org/docs/stable/distributions.html#normal
from torch.distributions.normal import Normal

m = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
m.sample()

tensor([1.5410])

In [4]:
wave = dataset.get_meta_train_batch(task_batch_size=T)[0]
samples = wave.get_samples()
x = samples["input"]
y_true = samples["output"]
x = torch.tensor(x[:,None])
y_true = torch.tensor(y_true[:,None])

In [5]:
x,y = get_samples_in_good_format(wave)
x

tensor([[-2.1909],
        [ 4.1874],
        [ 1.8886],
        [ 3.8937],
        [ 2.8011],
        [-3.1322],
        [-0.9134],
        [-4.8684],
        [ 1.8384],
        [ 4.4018]])

In [13]:
model = Prob_Neural_Network()
mu, sigma = model(x)
sigma += 1e-18 # avoid exact zero
m = Normal(mu, sigma)
-m.log_prob(y)

tensor([[-4.4239e-01],
        [ 3.9643e+35],
        [ 3.1114e+35],
        [ 5.8735e+35],
        [ 9.0731e+35],
        [ 1.8262e+00],
        [ 9.5487e+00],
        [ 7.0673e-02],
        [ 2.7476e+35],
        [ 2.7229e+35]], grad_fn=<NegBackward0>)

In [12]:
0.0000000000001

1e-13

In [13]:
m.log_prob(wave.get_samples()['output'][0])

tensor([-1.2675])

In [ ]:
importlib.reload(analysis_utils)

num_k_shots = 10
K = 10
num_eval=100
file_tag = "baseline_maml"
res = analysis_utils.k_shot_evaluation(model, dataset, num_k_shots=num_k_shots, K=K, num_eval=num_eval,
                        file_tag=file_tag)